In [66]:
import pymongo
import pandas as pd

In [7]:
game_dict= { 'NASA': {'user1':10,
                    'user2':7,
                    'user3':2},
           'STAR WARS': {'user2':5,
                        'user4':10,
                        'user5':2},
            'Mario': {'user1': 10,
                     'user3':8,
                     'user6':2}}

In [8]:
game_dict['STAR WARS']

{'user2': 5, 'user4': 10, 'user5': 2}

In [27]:
def flatten_game_dict(game_dict):
    for (game_id, user_score_dict) in game_dict.items():
        for (username, score) in user_score_dict.items():
            yield {'game_id': game_id, 'username': username, 'score': score}
            

In [30]:
flatten_game_dict(game_dict=game_dict)

<generator object flatten_game_dict at 0x104427780>

In [31]:
mc=pymongo.MongoClient()

In [32]:
db=mc['practice_db']

In [33]:
practice=db['practice']

In [41]:
flattened = flatten_game_dict(game_dict=game_dict)

In [42]:
for thing in flattened:
    practice.insert_one(thing)

In [43]:
practice.find_one()

{'_id': ObjectId('5c06d1d5933dea14d5c2aad3'),
 'game_id': 'NASA',
 'username': 'user1',
 'score': 10}

In [60]:
def load_reviews_for_game(key, game_id):
    found_reviews = practice.find({key: game_id})
    return found_reviews

In [65]:
load_reviews_for_game('username', 'user1').distinct('game_id')

['NASA', 'Mario']

In [67]:
df=pd.DataFrame(list(practice.find()))

In [72]:
df

,_id,game_id,score,username
0,5c06d1d5933dea14d5c2aad3,NASA,10,user1
1,5c06d1d6933dea14d5c2aad4,NASA,7,user2
2,5c06d1d6933dea14d5c2aad5,NASA,2,user3
3,5c06d1d6933dea14d5c2aad6,STAR WARS,5,user2
4,5c06d1d6933dea14d5c2aad7,STAR WARS,10,user4
5,5c06d1d6933dea14d5c2aad8,STAR WARS,2,user5
6,5c06d1d6933dea14d5c2aad9,Mario,10,user1
7,5c06d1d6933dea14d5c2aada,Mario,8,user3
8,5c06d1d6933dea14d5c2aadb,Mario,2,user6


In [83]:
for use in df.username.unique():
    print(use)

user1
user2
user3
user4
user5
user6


In [84]:
df[df.username=='user1']

,_id,game_id,score,username
0,5c06d1d5933dea14d5c2aad3,NASA,10,user1
6,5c06d1d6933dea14d5c2aad9,Mario,10,user1


In [85]:
df=df.pivot(index='username', columns='game_id', values='score' )

In [86]:
df

game_id,Mario,NASA,STAR WARS
username,,,
user1,10.0,10.0,NaN
user2,NaN,7.0,5.0
user3,8.0,2.0,NaN
user4,NaN,NaN,10.0
user5,NaN,NaN,2.0
user6,2.0,NaN,NaN
